# Azure チャット補完の例

この例では、Azure OpenAI サービスを使用したチャット補完について説明します。また、コンテンツフィルタリングに関する情報も含まれています。

## セットアップ

まず、必要な依存関係をインストールし、使用するライブラリをインポートします。

In [ ]:
! pip install "openai>=1.0.0,<2.0.0"
! pip install python-dotenv

In [ ]:
import os
import openai
import dotenv

dotenv.load_dotenv()

### 認証

Azure OpenAI サービスは、APIキーとAzure Active Directoryトークン認証情報を含む複数の認証メカニズムをサポートしています。

In [2]:
use_azure_active_directory = False  # Set this flag to True if you are using Azure Active Directory

#### APIキーを使用した認証

OpenAI SDKを*Azure APIキー*を使用するように設定するには、`api_key`をエンドポイントに関連付けられたキーに設定する必要があります（このキーは[Azure Portal](https://portal.azure.com)の*「リソース管理」*の下にある*「キーとエンドポイント」*で確認できます）。リソースのエンドポイントもここで確認できます。

In [13]:
if not use_azure_active_directory:
    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
    api_key = os.environ["AZURE_OPENAI_API_KEY"]

    client = openai.AzureOpenAI(
        azure_endpoint=endpoint,
        api_key=api_key,
        api_version="2023-09-01-preview"
    )

#### Azure Active Directoryを使用した認証
次に、Azure Active Directoryを使用して認証する方法を見てみましょう。まず、`azure-identity`ライブラリをインストールします。このライブラリは、認証に必要なトークン認証情報を提供し、`get_bearer_token_provider`ヘルパー関数を通じてトークン認証情報プロバイダーの構築を支援します。`AzureOpenAI`に静的トークンを提供するよりも`get_bearer_token_provider`を使用することが推奨されます。なぜなら、このAPIは自動的にトークンをキャッシュし、更新してくれるからです。

Azure OpenAIでAzure Active Directory認証を設定する方法の詳細については、[ドキュメント](https://learn.microsoft.com/azure/ai-services/openai/how-to/managed-identity)を参照してください。

In [ ]:
! pip install "azure-identity>=1.15.0"

In [5]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

if use_azure_active_directory:
    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]

    client = openai.AzureOpenAI(
        azure_endpoint=endpoint,
        azure_ad_token_provider=get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"),
        api_version="2023-09-01-preview"
    )

> 注意: AzureOpenAIは、以下の引数が提供されていない場合、対応する環境変数から推論します：

- `api_key` は `AZURE_OPENAI_API_KEY` から
- `azure_ad_token` は `AZURE_OPENAI_AD_TOKEN` から
- `api_version` は `OPENAI_API_VERSION` から
- `azure_endpoint` は `AZURE_OPENAI_ENDPOINT` から

## デプロイメント

このセクションでは、チャット補完を作成するために使用できるGPTモデルのデプロイメントを作成します。

### デプロイメント: Azure OpenAI Studioで作成する
チャット補完で使用するモデルをデプロイしましょう。https://portal.azure.com にアクセスし、Azure OpenAIリソースを見つけて、Azure OpenAI Studioに移動します。「Deployments」タブをクリックし、チャット補完に使用したいモデルのデプロイメントを作成します。モデルに付けるデプロイメント名は、以下のコードで使用されます。

In [4]:
deployment = "" # Fill in the deployment name from the portal here

## チャット補完の作成

それでは、作成したクライアントを使用してチャット補完を作成しましょう。

In [ ]:
# For all possible arguments see https://platform.openai.com/docs/api-reference/chat-completions/create
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Knock knock."},
        {"role": "assistant", "content": "Who's there?"},
        {"role": "user", "content": "Orange."},
    ],
    temperature=0,
)

print(f"{response.choices[0].message.role}: {response.choices[0].message.content}")

### ストリーミングチャット補完の作成

レスポンスをストリーミングすることもできます。

In [ ]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Knock knock."},
        {"role": "assistant", "content": "Who's there?"},
        {"role": "user", "content": "Orange."},
    ],
    temperature=0,
    stream=True
)

for chunk in response:
    if len(chunk.choices) > 0:
        delta = chunk.choices[0].delta

        if delta.role:
            print(delta.role + ": ", end="", flush=True)
        if delta.content:
            print(delta.content, end="", flush=True)

### コンテンツフィルタリング

Azure OpenAIサービスには、プロンプトと完了レスポンスのコンテンツフィルタリングが含まれています。コンテンツフィルタリングとその設定方法について詳しくは[こちら](https://learn.microsoft.com/azure/ai-services/openai/concepts/content-filter)をご覧ください。

プロンプトがコンテンツフィルターによってフラグ付けされた場合、ライブラリは`content_filter`エラーコードを持つ`BadRequestError`例外を発生させます。そうでない場合は、レスポンスの`prompt_filter_results`と`content_filter_results`にアクセスして、コンテンツフィルタリングの結果とどのカテゴリがフラグ付けされたかを確認できます。

#### コンテンツフィルターによってフラグ付けされたプロンプト

In [ ]:
import json

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "<text violating the content policy>"}
]

try:
    completion = client.chat.completions.create(
        messages=messages,
        model=deployment,
    )
except openai.BadRequestError as e:
    err = json.loads(e.response.text)
    if err["error"]["code"] == "content_filter":
        print("Content filter triggered!")
        content_filter_result = err["error"]["innererror"]["content_filter_result"]
        for category, details in content_filter_result.items():
            print(f"{category}:\n filtered={details['filtered']}\n severity={details['severity']}")

### コンテンツフィルターの結果の確認

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What's the biggest city in Washington?"}
]

completion = client.chat.completions.create(
    messages=messages,
    model=deployment,
)
print(f"Answer: {completion.choices[0].message.content}")

# prompt content filter result in "model_extra" for azure
prompt_filter_result = completion.model_extra["prompt_filter_results"][0]["content_filter_results"]
print("\nPrompt content filter results:")
for category, details in prompt_filter_result.items():
    print(f"{category}:\n filtered={details['filtered']}\n severity={details['severity']}")

# completion content filter result
print("\nCompletion content filter results:")
completion_filter_result = completion.choices[0].model_extra["content_filter_results"]
for category, details in completion_filter_result.items():
    print(f"{category}:\n filtered={details['filtered']}\n severity={details['severity']}")